# **Using HST COS data for Lyα** (Sujay)
### We only use FUVM coadded spectra

In [3]:
import altair as alt
import corv
import numpy as np
import pandas as pd

from astropy.table import Table
from astroquery.gaia import Gaia as GQ
from astroquery.simbad import Simbad as SQ
from pathlib import Path
from scripts import make_bins_alt
from tqdm import tqdm
from typing import Tuple

In [24]:
cos_objs = pd.read_csv('data/hst_cos/STAR_WD_COS_new.info', sep=r'\s,\s+', engine='python', usecols=[1, 2, 3], names=['ID', 'FLAG', 'META'])
cos_objs = cos_objs[cos_objs.META.str.contains('DA')]
cos_objs = cos_objs[~cos_objs.META.str.contains('PULSAR')].ID

gaia_ids = {}
for obj in cos_objs:
    query = SQ.query_objectids(obj).to_pandas().ID.str.decode('utf-8')
    gaia_ids[int(query[query.str.contains('DR3')].iloc[0].replace('Gaia DR3 ', ''))] = obj

params = GQ.launch_job(f'SELECT source_id, radial_velocity, teff_gspphot, logg_gspphot FROM gaiadr3.gaia_source WHERE source_id in {tuple(gaia_ids.keys())}').get_results().to_pandas()
params['NAMES'] = params.SOURCE_ID.map(gaia_ids)
params.to_csv('data/hst_cos/cos_params.csv', index=False)

In [37]:
#make_bins_alt.read_nlte_spectrum('TYC-7218-934-1')
source_id = 'WD2359-324'
spec = Table.read(f'data/hst_cos/{source_id.replace(' ', '-')}_coadd_FUVM_final_lpALL.fits.gz').to_pandas()
params = pd.read_csv('data/hst_cos/cos_params_updated.csv').query('NAMES == @source_id')
(params.RV.iloc[0], params.teff.iloc[0], params.logg.iloc[0]), spec.WAVE.values, spec.FLUX.values, 1 / (1e-6 + spec.ERROR.values)**2

((80.42913256991254, 14267.137705522417, 8.508031931608903),
 array([ 892.47976   ,  892.48971959,  892.49967918, ..., 1479.64736241,
        1479.657322  , 1479.66728159]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1.e+12, 1.e+12, 1.e+12, ..., 1.e+12, 1.e+12, 1.e+12]))

In [40]:
pd.read_csv('data/hst_cos/cos_params_updated.csv').dropna()


,Unnamed: 0,NAMES,SOURCE_ID,teff,logg,RV,e_RV
0,1,WD0106-328,5026963661794939520,15970.013916,8.356895,54.938465,2.639491
1,5,HE0308-2305,5075443981321647744,14864.825955,8.800420,74.030193,3.090362
2,7,HE0358-5127,4828757999190736128,14643.549939,8.615914,41.266260,2.554262
3,10,HE0403-4129,4842194473663089920,14499.999679,8.580147,26.898799,3.720792
4,12,HE0403-4129,4842194473663089920,14708.933844,8.835340,26.612462,4.375297
...,...,...,...,...,...,...,...
160,215,WD2350-248,2338275136195124864,27847.797871,8.766381,55.439573,3.869807
161,216,WD2350-248,2338275136195124864,27709.574772,8.784537,47.365674,4.582192
162,217,WD2359-324,2313582750735435776,14267.137706,8.508032,80.429133,4.380575
163,218,WD2359-324,2313582750735435776,13999.969864,8.062599,72.593497,6.068420


In [41]:
np.load('data_linecenters.npy').shape

(3, 4, 3, 20)